In [1]:
!pip install pandas

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/users_dataset.csv')
df.head()    # Displays the first 5 rows by default
df.info()      # Provides a summary of the DataFrame
df.describe()  # Provides statistical summaries for numeric columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userid  5000 non-null   int64 
 1   name    5000 non-null   object
 2   age     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


,userid,age
count,5000.000000,5000.000000
mean,2500.500000,58.407800
std,1443.520003,23.722817
min,1.000000,18.000000
25%,1250.750000,38.000000
50%,2500.500000,58.000000
75%,3750.250000,79.000000
max,5000.000000,99.000000


In [5]:
%%javascript
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min'
    }
});

<IPython.core.display.Javascript object>

In [9]:
from IPython.core.display import display, HTML

def display_d3(js_code):
    template = """
    <div id="container"></div>
    <script src="https://d3js.org/d3.v5.min.js"></script>
    <script>
    {0}
    </script>
    """.format(js_code)

    display(HTML(template))


In [21]:
ages_string = str(ages)
js_histogram_interactive_enhanced = f"""
var data = {ages_string};

var svg = d3.select("#container").append("svg")
    .attr("width", 600)
    .attr("height", 500)
    .append("g")
    .attr("transform", "translate(50, 20)");

var histogram = d3.histogram()
    .value(function(d) {{ return d; }})
    .domain([0, 100])
    .thresholds(50);

var bins = histogram(data);

var x = d3.scaleLinear()
    .range([0, 500])
    .domain([0, 100]);

var y = d3.scaleLinear()
    .range([450, 0])
    .domain([0, d3.max(bins, function(d) {{ return d.length; }})]);

var bars = svg.selectAll(".bar")
    .data(bins)
    .enter().append("rect")
    .attr("class", "bar")
    .attr("x", function(d) {{ return x(d.x0) + 1; }})
    .attr("y", function(d) {{ return y(d.length); }})
    .attr("width", function(d) {{ return x(d.x1) - x(d.x0) - 1; }})
    .attr("height", function(d) {{ return 450 - y(d.length); }})
    .attr("fill", "steelblue")
    .attr("rx", 3)  // Rounded corners
    .attr("ry", 3)
    .on("mouseover", function(d) {{
        d3.select(this)
            .attr("fill", "orange")
            .attr("y", y(d.length) - 5)
            .attr("height", 455 - y(d.length))
            .style("cursor", "pointer");
        svg.append("text")
            .attr("class", "tooltip")
            .attr("x", function() {{ return x(d.x0) + (x(d.x1) - x(d.x0)) / 2; }})
            .attr("y", function() {{ return y(d.length) - 10; }})
            .attr("text-anchor", "middle")
            .attr("pointer-events", "none")
            .text(function() {{ return d.x0 + "-" + d.x1 + ": " + d.length; }});
    }})
    .on("mouseout", function(d) {{
        d3.select(this)
            .attr("fill", "steelblue")
            .attr("y", y(d.length))
            .attr("height", 450 - y(d.length));
        d3.selectAll(".tooltip").remove();
    }});

// Add the X Axis
svg.append("g")
    .attr("transform", "translate(0," + 450 + ")")
    .call(d3.axisBottom(x));

// Add the Y Axis
svg.append("g")
    .call(d3.axisLeft(y));
"""

display_d3(js_histogram_interactive_enhanced)


In [16]:
# Age Groups
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
age_groups = df['age_group'].value_counts().sort_index().to_dict()

# Top Names
top_names = df['name'].value_counts().head(10).to_dict()


In [19]:
age_groups_string = str(age_groups)
js_age_groups_interactive = f"""
var data = {age_groups_string};

var svg = d3.select("#container").append("svg")
    .attr("width", 500)
    .attr("height", 500);

var x = d3.scaleBand()
    .range([0, 500])
    .domain(Object.keys(data))
    .padding(0.1);

var y = d3.scaleLinear()
    .range([500, 0])
    .domain([0, d3.max(Object.values(data))]);

var bars = svg.selectAll(".bar")
    .data(Object.entries(data))
    .enter().append("rect")
    .attr("class", "bar")
    .attr("x", function(d) {{ return x(d[0]); }})
    .attr("width", x.bandwidth())
    .attr("y", function(d) {{ return y(d[1]); }})
    .attr("height", function(d) {{ return 500 - y(d[1]); }})
    .on("mouseover", function(d) {{
        d3.select(this).style("fill", "orange");
        svg.append("text")
            .attr("class", "value")
            .attr("x", function() {{ return x(d[0]) + x.bandwidth() / 2; }})
            .attr("y", function() {{ return y(d[1]) - 10; }})
            .attr("text-anchor", "middle")
            .text(function() {{ return d[1]; }});
    }})
    .on("mouseout", function() {{
        d3.select(this).style("fill", "steelblue");
        d3.selectAll(".value").remove();
    }});
"""

display_d3(js_age_groups_interactive)


In [23]:
top_names_string = str(top_names)
js_pie_chart = f"""
var data = {top_names_string};

var width = 600;
var height = 500;
var radius = Math.min(width, height) / 2;

var svg = d3.select("#container")
    .append("svg")
    .attr("width", width)
    .attr("height", height)
    .append("g")
    .attr("transform", "translate(" + width / 2 + "," + height / 2 + ")");

var color = d3.scaleOrdinal(d3.schemeCategory10);

var pie = d3.pie().value(function(d) {{ return d[1]; }});
var arc = d3.arc().innerRadius(0).outerRadius(radius);

var g = svg.selectAll(".arc")
    .data(pie(Object.entries(data)))
    .enter().append("g")
    .attr("class", "arc");

g.append("path")
    .attr("d", arc)
    .style("fill", function(d, i) {{ return color(i); }})
    .on("mouseover", function(d) {{
        d3.select(this)
            .transition()
            .duration(100)
            .attr("d", d3.arc().innerRadius(0).outerRadius(radius + 10));
        svg.append("text")
            .attr("class", "tooltip")
            .attr("dy", ".35em")
            .style("text-anchor", "middle")
            .attr("pointer-events", "none")
            .text(d.data[0] + ": " + d.data[1]);
    }})
    .on("mouseout", function(d) {{
        d3.select(this)
            .transition()
            .duration(100)
            .attr("d", arc);
        d3.selectAll(".tooltip").remove();
    }});
"""

display_d3(js_pie_chart)
